# Introduction à SQL

---
## Exercice 1 et 2 - Dragons !

In [21]:
    dragons = [
        ("Sméagol", "M", 152, 1857, 1, "macho"),
        ("Birdurh", "M", 258, 4787, 0, "timide"),
        ("Négueth", "F", 128, 1581, 1, "sincère"),
        ("MissToc", "F", 183, 2781, 0, "superflu"),
        ("Bolong", "M", 213, 2751, 1, "macho"),
        ("Miloch", "M", 83, 718, 1, "timide"),
        ("Nessie", "M", 168, 1721, 0, "absent"),
        ("Tarak", "F", 123, 851, 1, "timide"),
        ("Solong", "M", 173, 1481, 1, "sincère")        
    ]
    
    aimé = [
        ("Sméagol", "Négueth", "passionnément"),
        ("Birdurh", "Négueth", "beaucoup"),
        ("Négueth", "Miloch", "à la folie"),
        ("Bolong", "Négueth", "à la folie"),
        ("Tarak", "Bolong", "un peu"),
        ("Solong", "Tarak", "beaucoup"),    
    ]
    
    nourritures = [
        ("pomme", 7),
        ("cacahuète", 10),
        ("orange", 25),
        ("oeuf", 15),
        ("ver", 3),
        ("poisson", 45),
        ("humain", 215),
    ]
    
    repas = [
        ("Sméagol", "cacahuète", 1000),
        ("Sméagol", "pomme", 16),
        ("Sméagol", "humain", 2),
        ("Birdurh", "oeuf", 26),
        ("Négueth", "oeuf", 1),
        ("Négueth", "orange", 6),
        ("Négueth", "humain", 1),
        ("Miloch", "ver", 95),
        ("Miloch", "humain", 3),
        ("Nessie", "poisson", 42),
        ("Tarak", "pomme", 10),
        ("Tarak", "orange", 10),
        ("Solong", "oeuf", 13),
        ("Solong", "poisson", 7),
        ("Solong", "orange", 1),
        ("Solong", "humain", 2),
        ("Bolong", "humain", 5),        
    ]

In [22]:
import sqlite3

with sqlite3.connect('dragons.db') as conn:
    cur = conn.cursor()
    
    cur.execute("PRAGMA  foreign_keys = ON")
    conn.commit()
    
    # Nettoyage et effacement de la base de données existante
    cur.execute("DROP TABLE IF EXISTS Repas")
    cur.execute("DROP TABLE IF EXISTS Nourritures")
    cur.execute("DROP TABLE IF EXISTS Aime")
    cur.execute("DROP TABLE IF EXISTS Dragons")
    conn.commit()

    # Création des tables
    cur.execute("""
    CREATE TABLE Dragons (
        Nom TEXT PRIMARY KEY,
        Sexe TEXT CHECK (Sexe = "M" OR Sexe = "F"),
        Longueur INTEGER,
        NombreEcailles INTEGER,
        CracheDuFeu INTEGER CHECK (CracheDuFeu = 0 OR CracheDuFeu = 1),
        ComportementAmoureux TEXT
    )
    """)
    
    cur.execute("""
    CREATE TABLE Aime (
        DragonAimant TEXT,
        DragonAime TEXT,
        Force TEXT,
        PRIMARY KEY (DragonAimant, DragonAime),
        FOREIGN KEY (DragonAimant) REFERENCES Dragons(Nom),
        FOREIGN KEY (DragonAime) REFERENCES Dragons(Nom)
    )
    """)
    
    cur.execute("""
    CREATE TABLE Nourritures (
        Produit TEXT PRIMARY KEY,
        Calories INTEGER
    )
    """)    
    
    cur.execute("""
    CREATE TABLE Repas (
        Dragon TEXT,
        Produit TEXT,
        Quantite INTEGER,
        FOREIGN KEY (Dragon) REFERENCES Dragons(Nom),
        FOREIGN KEY (Produit) REFERENCES Nourritures(Produit)
    )
    """)
    
    conn.commit()
    
    # Remplissage de la table Dragons:
    cur.executemany("INSERT INTO Dragons VALUES(?, ?, ?, ?, ?, ?)", dragons)
    cur.executemany("INSERT INTO Aime VALUES(?, ?, ?)", aimé)
    cur.executemany("INSERT INTO Nourritures VALUES(?, ?)", nourritures)
    cur.executemany("INSERT INTO Repas VALUES(?, ?, ?)", repas)
    conn.commit()

---
## Exercice 3 - Départements

In [25]:
import csv

# On crée un dictionnaire associant à chaque code de département
# un autre dictionnaire contenant les clés 'nom' et 'région'.
départements = {}
# En parallèle, on crée un dictionnaire associant à chaque nom de
# région la liste de ses départements
régions = {}
with open('departements/departements.csv') as csvfile:
    reader = csv.reader(csvfile)
    for ligne in reader:
        code, nom, région = ligne
        départements[code] = {'nom': nom, 'région': région}
        if région in régions:
            régions[région].append(code)
        else:
            régions[région] = [code]

# On crée un dictionnaire associant à chaque code de département
# La liste (tableau) des codes des départements limitrophes.
limitrophes = {}
# On gère la Corse à la main, car elle n'est pas mentionnée 
# dans le fichier limitrophes.csv
limitrophes['2A'] = ['2B']
limitrophes['2B'] = ['2A']
with open('departements/limitrophes.csv') as csvfile:
    reader = csv.reader(csvfile)
    for ligne in reader:
        code = ligne[0]
        # Le code est-il un des codes de départements 
        # stockés plus haut ? (pour éviter les arrondissements
        # de Paris qui sont dans le fichier csv) ?
        if code in départements:
            # Oui: on cherche les départements limitrophes
            limitrophes[code] = []
            for i in range(1, 9):
                # ligne[i] est-il un département valide ?
                # (on veut éviter les entrées '\xa0' marquant
                # un voisin vide)
                if ligne[i] in départements:
                    # Oui: on le rajoute
                    limitrophes[code].append(ligne[i])

In [31]:
import sqlite3
with sqlite3.connect('departements.db') as conn:
    cur = conn.cursor()

    # On commence par créer (après les avoir éventuellement 
    # détruites) les trois tables:
    cur.executescript("""
    PRAGMA foreign_keys = ON;

    DROP TABLE IF EXISTS Limitrophes;
    DROP TABLE IF EXISTS Departements;
    DROP TABLE IF EXISTS Regions;
    
    CREATE TABLE Regions (
        id_region INTEGER PRIMARY KEY,
        nom TEXT
    );
    
    CREATE TABLE Departements (
        code TEXT,
        nom TEXT,
        id_region INTEGER,
        FOREIGN KEY(id_region) REFERENCES Regions(id_region)
    );
    
    CREATE TABLE Limitrophes (
        code1 TEXT,
        code2 TEXT,
        PRIMARY KEY(code1, code2),
        FOREIGN KEY(code1) REFERENCES Departements(code),
        FOREIGN KEY(code2) REFERENCES Departements(code),
        CHECK (code1 < code2)
    );
    """)
    
    conn.commit()
    
    # On remplit la table Regions:
    # On crée en parallèle un dictionnaire qui permet de
    # retrouver facilement l'id d'une région
    régions_id = {}
    id_région = 1
    for r in régions:
        cur.execute("""
        INSERT INTO Regions VALUES ( ?, ? )
        """, (id_région, r))
    
        régions_id[r] = id_région
        id_région += 1
    conn.commit()
    
    # Puis la table Departements:
    for code in départements:
        nom = départements[code]['nom']
        région = départements[code]['région']
        
        id_région = régions_id[région]
        cur.execute("""
        INSERT INTO Departements VALUES (?, ?, ?)
        """, (code, nom, id_région))
    conn.commit()
    
    # Finalement, on remplit la table Limitrophes:
    for code1 in limitrophes:
        for code2 in limitrophes[code1]:
            print(repr(code1), repr(code2))
            cur.execute("""
            INSERT INTO Limitrophes VALUES ( ?, ? )
            """, (code1, code2))
    conn.commit()


'2A' '2B'


OperationalError: foreign key mismatch - "Limitrophes" referencing "Departements"

---
## Exercice 4 - Pokemons, le retour

Le fichier `pokemons.csv` dans le répertoire `pokemons` contient de nombreuses données sur de non moins nombreux pokemons.

1. Lisez et chargez en mémoire ce fichier à l'aide d'un objet `DictReader` de la librairie `csv`. Profitez-en pour examiner la structure d'un enregistrement.
1. Nous souhaitons créer une base de donnée à l'aide d'une _partie_ des données extraites de ce fichier csv. Créez et spécifiez le schéma de cette base, sachant que:
  * Un  pokémon est caractérisé par son nom et son numéro d’identification unique dans le pokedex. Il dispose d’un type primaire et d’un type secondaire (ce dernier pouvant être vide), de caractéristiques numériques (points de vie, attaque, défense, attaques et défenses spéciales, vitesse), et de mouvements (dont le nombre peut varier... ce qui nécessite une relation à part). On doit pouvoir retrouver le nom de l'image de chaque pokemon (mais l'image elle-même n'est pas stockée dans la base, bien que cela soit possible à l'aide du type `BLOB`);
  * Les caractéristiques numériques pourraient être placées dans leur propre relation;
  * Les types et les mouvements (_abilities_) ne devraient jamais être saisis plus d'une fois: ils leur faut donc leur propre relation, afin d'offrir le maximum de souplesse et de vérification de contraintes d'intégrité;
1. Créez les tables avec `sqlite3`, basées sur votre schéma.
1. Remplissez ces tables à partir de vos données.